In [1]:
import ast
import jedi

project_dir = "src"
file_name = "jedi_test.py"

file_contents = open(f"{project_dir}/{file_name}").read()

project = jedi.Project(path=project_dir)
script = jedi.Script(file_contents, path=file_name, project=project)
tree = ast.parse(file_contents)

function_definitions = set()

for node in ast.walk(tree):
    if node.__class__.__name__ == 'Call':
        new_function_definitions = script.goto(
            node.lineno, 
            node.col_offset,
            follow_imports=True,
            follow_builtin_imports=True,
        )
        function_definitions.update(new_function_definitions)

print(function_definitions)

for function_definition in function_definitions:
    start_line, _ = function_definition.get_definition_start_position()
    end_line, _ = function_definition.get_definition_end_position()
    module_contents = open(function_definition.module_path).read()
    print("\n".join(module_contents.split("\n")[max(0, start_line - 1): end_line]))


{<Name full_name='helpers.test2.a_func', description='def a_func'>}
def a_func():
    a = 1
    b = 2 * a
    c = a * foo() + b * 3
    print(b, c)


In [9]:
import ast
import os
import sys
import jedi
from jedi.api.classes import Name

project_dir = "../.."
file_name = "sweepai/api.py"
min_line = 101
max_line = 110

file_full_path = f"{project_dir}/{file_name}"

file_contents = open(file_full_path).read()

project_absolute_path = os.path.abspath(project_dir)

BUILTIN_MODULES = [builtin_module_name.strip("_") for builtin_module_name in sys.builtin_module_names] + ["builtins"]

project = jedi.Project(path=project_absolute_path)
script = jedi.Script(file_contents, path=file_name, project=project)
tree = ast.parse(file_contents)

function_definitions: set[Name] = set()

for node in ast.walk(tree):
    if node.__class__.__name__ == 'Call':
        # if not min_line <= node.lineno <= max_line:
        #     continue
        new_function_definitions = script.goto(
            node.lineno, 
            node.col_offset,
            follow_imports=True,
            follow_builtin_imports=True,
        )
        for function_definition in new_function_definitions:
            print(function_definition.module_path)
            if function_definition.full_name and any(function_definition.full_name.startswith(builtin_module) for builtin_module in BUILTIN_MODULES):
                continue
            if function_definition.type != "function" and function_definition.type != "statement":
                continue
            function_definitions.add(function_definition)

# for function_definition in function_definitions:
#     start_line, _ = function_definition.get_definition_start_position()
#     end_line, _ = function_definition.get_definition_end_position()
#     if os.path.exists(function_definition.module_path):
#         module_contents = open(function_definition.module_path).read()
#     else:
#         module_contents = open(file_full_path).read()
#     print("\n".join(module_contents.split("\n")[max(0, start_line - 1): end_line]))


/home/kevin/sweep/sweepai/logn/logn.py
/home/kevin/.cache/pypoetry/virtualenvs/sweepai-u_CIt3kb-py3.10/lib/python3.10/site-packages/fastapi/applications.py
/usr/lib/python3.10/tracemalloc.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/usr/lib/python3.10/threading.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/sweepai/logn/logn.py
/home/kevin/sweep/sweepai/handlers/on_check_suite.py
/home/kevin/sweep/sweepai/logn/logn.py
/home/kevin/.cache/pypoetry/virtualenvs/sweepai-u_CIt3kb-py3.10/lib/python3.10/site-packages/jedi/third_party/typeshed/stdlib/2and3/time.pyi
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/home/kevin/sweep/tests/notebooks/sweepai/api.py
/usr/lib/pytho

In [104]:
code = """
def foo(a: int):
    return bar(str(a))

def bar(b):
    return b
"""

script = jedi.Script(code)
# script.infer(3, 16)[0].type
script.infer(3, 16)[0].rtype

AttributeError: 'Name' object has no attribute 'rtype'

In [ ]:
code = """
def foo(a: int):
    return bar(str(a))

def bar(b):
    return b
"""

script = jedi.Script(code)
script.infer(3, 16)[0].rtype